<a href="https://colab.research.google.com/github/sibivel/finance/blob/main/Portfolio_Rebalancing_Jupyter_Notebook_(Colab_Ready).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#
#          PORTFOLIO REBALANCING & INVESTMENT ALLOCATION TOOL
#                       (Google Colab Version)
#
# ==============================================================================

# --- DESCRIPTION ---
# This Jupyter Notebook helps you decide where to invest new cash to bring your
# portfolio closer to your desired allocations. It uses a mathematical
# optimization solver to find the most effective allocation for each category.
#
# --- HOW TO USE ---
# 1. Run the first code cell. It will try to find 'Portfolio.csv'. If not found,
#    it will ask you to upload your Fidelity positions CSV file.
# 2. Uploaded files are saved as 'Portfolio.csv' for future runs.
# 3. Fill out the configuration sections below (Sections A, B, and C).
# 4. Run the remaining cells in the notebook from top to bottom.
# 5. The final output will be two separate strategic plans: one for Asset
#    Classes and one for Sectors.

import pandas as pd
import io
import numpy as np
from scipy.optimize import minimize
import os

In [ ]:
# ==============================================================================
# SECTION FOR GOOGLE COLAB: FILE UPLOAD & PERSISTENCE
# ==============================================================================
# Run this cell to load your data. It will look for 'Portfolio.csv' first.

DEFAULT_FILENAME = 'Portfolio.csv'
file_content = None

try:
    # Check if the file already exists in the Colab environment
    if os.path.exists(DEFAULT_FILENAME):
        print(f"Found existing '{DEFAULT_FILENAME}'. Reading file...")
        with open(DEFAULT_FILENAME, 'r', encoding='utf-8-sig') as f:
            file_content = f.read()
        print("File loaded successfully. You can now run the rest of the notebook.")
    else:
        # If the file doesn't exist, prompt for upload
        from google.colab import files
        print(f"'{DEFAULT_FILENAME}' not found. Please upload your Fidelity Positions CSV file:")
        uploaded = files.upload()

        if uploaded:
            # Get the name and content of the uploaded file
            file_name = next(iter(uploaded))
            # Use 'utf-8-sig' to automatically handle the BOM (Byte Order Mark)
            file_content = uploaded[file_name].decode('utf-8-sig')

            # Save the uploaded file with the default name for future runs
            with open(DEFAULT_FILENAME, 'w', encoding='utf-8-sig') as f:
                f.write(file_content)
            print(f"\nSuccessfully uploaded and saved as '{DEFAULT_FILENAME}'. You can now run the rest of the notebook.")
        else:
            print("\nOperation cancelled. No file was uploaded.")

except ImportError:
    # This block will run if not in a Colab environment
    print("This notebook is not running in Google Colab.")
    print(f"Please place your CSV as '{DEFAULT_FILENAME}' in the same directory.")
    if os.path.exists(DEFAULT_FILENAME):
         with open(DEFAULT_FILENAME, 'r', encoding='utf-8-sig') as f:
            file_content = f.read()


'Portfolio.csv' not found. Please upload your Fidelity Positions CSV file:


Saving Portfolio_Positions_Nov-08-2025.csv to Portfolio_Positions_Nov-08-2025.csv

Successfully uploaded and saved as 'Portfolio.csv'. You can now run the rest of the notebook.


In [ ]:

# ==============================================================================
# SECTION A: CORE CONFIGURATION
# ==============================================================================
# --- Step 1: Define the amount of new cash you want to invest ---
CASH_TO_INVEST = 10000.00


# ==============================================================================
# SECTION B: DEFINE YOUR INVESTMENT GOALS
# ==============================================================================

# --- Step 2: Define your target Asset Class allocation ---
# These are your high-level geographic or strategy goals.
# The percentages MUST add up to 1.0 (100%).
target_asset_class_allocation = {
    'US': 0.80,
    'International': 0.20,
    'Bonds': 0,
}

# --- Step 3: Define your target Sector allocation ---
# These are your goals for industry exposure across your entire portfolio.
# The percentages MUST add up to 1.0 (100%).
target_sector_allocation = {
    'Technology': 0.30,
    'Healthcare': 0.1,
    'Financials': 0.1,
    'Broad Market': 0.30, # non-tech since most broad market is highly tech already.
    'Energy': 0.05,
    'Real Estate': 0.1,
    'Other': 0.05,
}


# ==============================================================================
# SECTION C: CATEGORIZE YOUR HOLDINGS
# ==============================================================================

# --- Step 4: Map each of your tickers to its weighted categories ---
# This is the most important step. Each ticker maps to a LIST of allocations.
# Format: 'TICKER': [('Asset Class', 'Sector', weight)]
# The weights for each ticker MUST sum to 1.0.
# Use (None, None, 1.0) for any ticker you want to be completely ignored.
asset_and_sector_map = {
    # Ticker: [('Asset Class', 'Sector', weight)]
    'IXUS': [('International', 'Broad Market', 0.88), ('International', 'Technology', 0.12)],
    'SMH':  [('US', 'Technology', 1.0)],
    'QQQ':  [('US', 'Technology', 0.5), ('US', 'Broad Market', 0.5)],
    'IUSB':  [('Bonds', 'Other', 1.0)],
    'DIS':  [('US', 'Other', 1.0)],
    'ARKK': [('US', 'Technology', 1.0)],
    'AAPL': [('US','Technology', 1.0)],
    'BETZ': [('US', 'Other', 1.0)],
    'FHLC': [('US','Healthcare', 1.0)],
    'FNCL': [('US','Financials', 1.0)],
    'ICLN': [('US', 'Energy', 1.0)],
    'INDA': [('International', 'Broad Market', 1.0)],
    'ITOT': [('US', 'Broad Market', 0.7), ('US', 'Technology', 0.3)],
    'QCLN': [('US', 'Energy', 1.0)],
    'RBLX': [('US', 'Other', 1.0)],
    'REET': [('US', 'Real Estate', 0.7), ('International', 'Real Estate', 0.3)],
    'USRT': [('US', 'Real Estate', 1.0)],
    'VOO': [('US', 'Broad Market', 0.7), ('US', 'Technology', 0.3)],
    'BPAY': [('US', 'Financials', 0.70), ('International', 'Financials', 0.3)],
    'FFOPX': [(None, None, 1.0)], # This ticker will be ignored
}


# ==============================================================================
#
#          --- CORE LOGIC --- (No need to edit below this line)
#
# ==============================================================================

def load_and_clean_positions(file_content_string):
    """
    Loads the Fidelity CSV from a string of file content, cleans it,
    and returns a clean DataFrame with Symbol and Current Value.
    """
    if not file_content_string:
        print("--- ERROR: No file content to process. Please run the file upload cell first. ---")
        return None

    lines = file_content_string.splitlines(True)

    start_index = -1
    for i, line in enumerate(lines):
        if line.strip().startswith('Account Number,Account Name,'):
            start_index = i
            break

    if start_index == -1:
        print("--- ERROR: Could not find the required header row in your file. ---")
        return None

    csv_data = io.StringIO(''.join(lines[start_index:]))
    df = pd.read_csv(csv_data)

    df = df[['Symbol', 'Current Value']].copy()
    df.dropna(subset=['Symbol'], inplace=True)

    # Filter out cash-equivalent funds (like SPAXX**) which are not part of the investment portfolio
    df = df[~df['Symbol'].str.contains('\*\*')].copy()

    df['Current Value'] = df['Current Value'].replace({'\$': '', ',': ''}, regex=True).astype(float)
    df = df.groupby('Symbol')['Current Value'].sum().reset_index()

    return df

def run_single_category_optimization(current_category_values, target_allocation, cash_to_invest, total_initial_value):
    """
    A generalized optimizer that finds the best allocation for a single dimension
    (e.g., just for Asset Classes, or just for Sectors).
    """
    categories = sorted(list(target_allocation.keys()))
    current_values = current_category_values.reindex(categories).fillna(0)
    total_final_value = total_initial_value + cash_to_invest

    def objective_function(x):
        final_category_values = current_values + x * cash_to_invest
        final_allocation = final_category_values / total_final_value

        target_s = pd.Series(target_allocation, name='target')
        actual_s = final_allocation.rename('actual')

        comp = pd.concat([target_s, actual_s], axis=1).fillna(0)
        error = comp['target'] - comp['actual']

        return np.sum(error**2)

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = [(0, None) for _ in range(len(categories))]
    x0 = np.array([1 / len(categories)] * len(categories))

    result = minimize(objective_function, x0, method='SLSQP', bounds=bounds, constraints=constraints)

    if not result.success:
        print(f"--- WARNING: The optimizer may not have found a solution. Message: {result.message} ---")

    investment_plan = {category: amount for category, amount in zip(categories, result.x * cash_to_invest) if amount > 1.0}
    return investment_plan


def display_separate_results(current_asset_values, current_sector_values, asset_plan, sector_plan, target_assets, target_sectors, total_initial_value):
    """
    Displays the final analysis and the two separate investment plans.
    """
    print("\n" + "="*80)
    print("                    PORTFOLIO REBALANCING ANALYSIS")
    print("="*80 + "\n")

    # --- Function to generate allocation tables ---
    def generate_comparison_table(current_values, investment_plan, target_dict):
        cash_invested = sum(investment_plan.values())
        final_value = total_initial_value + cash_invested

        # Calculate "Before" state
        before_pct = (current_values / total_initial_value * 100).rename('Before %')

        # Calculate "After" state
        after_alloc = current_values.copy()
        for category, amount in investment_plan.items():
            if category in after_alloc.index:
                after_alloc[category] += amount
            else:
                after_alloc[category] = amount
        after_pct = (after_alloc / final_value * 100).rename('After %')

        # Combine into a final display table
        target_pct = (pd.Series(target_dict) * 100).rename('Target %')
        comparison_df = pd.concat([target_pct, before_pct, after_pct], axis=1).fillna(0)
        return comparison_df[['Target %', 'Before %', 'After %']].round(2)

    # --- Generate and Print Tables ---
    asset_comparison = generate_comparison_table(current_asset_values, asset_plan, target_assets)
    sector_comparison = generate_comparison_table(current_sector_values, sector_plan, target_sectors)

    print("--- Asset Class Allocation ---")
    print(asset_comparison.to_string())
    print("\n" + "-"*80 + "\n")

    print("--- Sector Allocation ---")
    print(sector_comparison.to_string())
    print("\n" + "="*80 + "\n")

    # --- Print Investment Plans ---
    def print_plan(plan, name):
        cash_invested = sum(plan.values())
        print(f"          RECOMMENDED INVESTMENT PLAN: {name}")
        print(f"         Total to Invest: ${cash_invested:,.2f}")
        print("-" * 50 + "\n")

        plan_df = pd.DataFrame(list(plan.items()), columns=[name, 'Amount to Invest'])

        if plan_df.empty:
            print(f"No investments recommended for {name}. This category is balanced.")
        else:
            plan_df = plan_df.sort_values(by='Amount to Invest', ascending=False).reset_index(drop=True)
            plan_df['Amount to Invest'] = plan_df['Amount to Invest'].map("${:,.2f}".format)
            print(plan_df.to_string(index=False))
        print("\n" + "="*80 + "\n")

    print_plan(asset_plan, 'Asset Class')
    print_plan(sector_plan, 'Sector')


# ==============================================================================
#                                 MAIN EXECUTION
# ==============================================================================
def run_main_logic():
    if 'file_content' not in globals() or not file_content:
        print("--- ERROR --- \nPlease run the first cell and upload your CSV file before running this cell.")
        return

    if not (round(sum(target_asset_class_allocation.values()), 5) == 1.0 and round(sum(target_sector_allocation.values()), 5) == 1.0):
        print("--- ERROR: Target allocation percentages in Section B do not add up to 1.0 (100%). Please correct them. ---")
        return

    positions = load_and_clean_positions(file_content)

    if positions is not None:
        # --- VALIDATION STEP 1: Check for uncategorized symbols ---
        all_symbols_in_portfolio = set(positions['Symbol'])
        all_symbols_in_map = set(asset_and_sector_map.keys())
        uncategorized_symbols = all_symbols_in_portfolio - all_symbols_in_map

        if uncategorized_symbols:
            print("\n--- FATAL ERROR: The following tickers from your portfolio are not categorized in Section C ---")
            print("Please add them to the 'asset_and_sector_map' and rerun the script.")
            for symbol in sorted(list(uncategorized_symbols)):
                print(f" - {symbol}")
            return

        # --- VALIDATION STEP 2: Check that weights for each ticker sum to 1.0 ---
        for ticker, allocations in asset_and_sector_map.items():
            total_weight = sum(weight for _, _, weight in allocations)
            if not np.isclose(total_weight, 1.0):
                print(f"\n--- FATAL ERROR: The weights for ticker '{ticker}' in Section C do not sum to 1.0. Current sum is {total_weight}. ---")
                return

        # --- VALIDATION STEP 3: Check for undefined categories ---
        mapped_asset_classes = {asset for allocations in asset_and_sector_map.values() for asset, _, _ in allocations if asset is not None}
        mapped_sectors = {sector for allocations in asset_and_sector_map.values() for _, sector, _ in allocations if sector is not None}

        undefined_assets = mapped_asset_classes - set(target_asset_class_allocation.keys())
        undefined_sectors = mapped_sectors - set(target_sector_allocation.keys())

        if undefined_assets:
            print("\n--- FATAL ERROR: The following Asset Classes are used in your map but not defined in your targets ---")
            for asset in sorted(list(undefined_assets)):
                print(f" - {asset}")
            return

        if undefined_sectors:
            print("\n--- FATAL ERROR: The following Sectors are used in your map but not defined in your targets ---")
            for sector in sorted(list(undefined_sectors)):
                print(f" - {sector}")
            return

        # --- Calculate current values based on weighted allocations ---
        current_asset_values = {cat: 0 for cat in target_asset_class_allocation}
        current_sector_values = {cat: 0 for cat in target_sector_allocation}

        for _, row in positions.iterrows():
            symbol = row['Symbol']
            total_value = row['Current Value']
            allocations = asset_and_sector_map.get(symbol)

            if allocations:
                for asset, sector, weight in allocations:
                    if asset is not None and asset in current_asset_values:
                        current_asset_values[asset] += total_value * weight
                    if sector is not None and sector in current_sector_values:
                        current_sector_values[sector] += total_value * weight

        current_asset_values = pd.Series(current_asset_values)
        current_sector_values = pd.Series(current_sector_values)
        total_initial_value = current_asset_values.sum() # Use sum of categorized assets as the base

        # --- Run two separate optimizations ---
        # 1. For Asset Classes
        asset_plan = run_single_category_optimization(
            current_asset_values, target_asset_class_allocation, CASH_TO_INVEST, total_initial_value
        )

        # 2. For Sectors
        sector_plan = run_single_category_optimization(
            current_sector_values, target_sector_allocation, CASH_TO_INVEST, total_initial_value
        )

        display_separate_results(
            current_asset_values,
            current_sector_values,
            asset_plan,
            sector_plan,
            target_asset_class_allocation,
            target_sector_allocation,
            total_initial_value
        )

run_main_logic()
